# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [42]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import time
import sys

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

// Add this as a new Markdown cell directly under "## Acquiring an API Token"

> Note: This notebook uses the Finnhub Stock API instead of IEX Cloud because IEX Cloud has been closed. More info: https://iexcloud.org/

In [3]:
from secretsRENAMED import FINNHUB_API_KEY

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



## Finnhub

In [4]:
symbol = 'AAPL'

pricing_api_url = f'https://finnhub.io/api/v1/quote?symbol={symbol}&token={FINNHUB_API_KEY}'
pricing_data = requests.get(pricing_api_url).json()
print(pricing_data)

market_cap_api_url = f'https://finnhub.io/api/v1/stock/profile2?symbol={symbol}&token={FINNHUB_API_KEY}'
market_cap_data = requests.get(market_cap_api_url).json()
print(market_cap_data)

{'c': 270.97, 'd': -2.7, 'dp': -0.9866, 'h': 273.88, 'l': 270.505, 'o': 272.86, 'pc': 273.67, 't': 1766437200}
{'country': 'US', 'currency': 'USD', 'estimateCurrency': 'USD', 'exchange': 'NASDAQ NMS - GLOBAL MARKET', 'finnhubIndustry': 'Technology', 'ipo': '1980-12-12', 'logo': 'https://static2.finnhub.io/file/publicdatany/finnhubimage/stock_logo/AAPL.png', 'marketCapitalization': 4003948.4439650117, 'name': 'Apple Inc', 'phone': '14089961010', 'shareOutstanding': 14776.35, 'ticker': 'AAPL', 'weburl': 'https://www.apple.com/'}


In [5]:
print("price: ")
print(pricing_data['c'])

print("market cap: ")
print(market_cap_data['marketCapitalization'])

price: 
270.97
market cap: 
4003948.4439650117


## Alpha Vantage

In [4]:
symbol = 'AAPL'
pricing_api_url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&apikey={ALPHA_VANTAGE_API_KEY}&symbol={symbol}&interval=60min'
pricing_data = requests.get(pricing_api_url).json()
print(pricing_data)

market_cap_api_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={ALPHA_VANTAGE_API_KEY}'
market_cap_data = requests.get(market_cap_api_url).json()
print(market_cap_data)

{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


In [4]:
print("price: ")
print(pricing_data['Time Series (60min)']['2023-12-08 16:00:00']['4. close'])

print("market cap: ")
print(market_cap_data['MarketCapitalization'])

{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}
{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [6]:
price = pricing_data['c']
print("price:", price)

market_cap = market_cap_data['marketCapitalization']/1000000
print("market cap:", market_cap, 'trillion')

price: 270.97
market cap: 4.003948443965012 trillion


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [7]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [8]:
new_row = pd.Series(
    [symbol, price, market_cap, 'N/A'],
    index=my_columns
)

pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,270.97,4.003948,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [80]:
final_dataframe = pd.DataFrame(columns = my_columns)
rows = []
for stock in stocks['Ticker']:
    try:
        pricing_api_url = f'https://finnhub.io/api/v1/quote?symbol={stock}&token={FINNHUB_API_KEY}'
        market_cap_api_url = f'https://finnhub.io/api/v1/stock/profile2?symbol={stock}&token={FINNHUB_API_KEY}'

        pricing_data = requests.get(pricing_api_url).json()
        if pricing_data.get('error'):
            raise Exception(f"Error fetching pricing data for {stock}: {pricing_data['error']}")

        market_cap_data = requests.get(market_cap_api_url).json()
        if market_cap_data.get('error'):
            raise Exception(f"Error fetching market cap data for {stock}: {market_cap_data['error']}")

        price = pricing_data.get('c') or 'N/A'
        market_cap = market_cap_data.get('marketCapitalization') or 'N/A'

        price = price if price != 0 else 'N/A'
        market_cap = market_cap / 1000 if isinstance(market_cap, (int, float)) and market_cap != 0 else 'N/A'

    except Exception as e:
        print(f"An error occurred for {stock}: {e}")
        price = 'N/A'
        market_cap = 'N/A'
    
    rows.append([stock, price, market_cap, 'N/A'])
    
    time.sleep(1)

final_dataframe = pd.DataFrame(rows, columns=my_columns)

In [81]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,138.42,39.208695,N/A
1,AAL,15.795,10.733006,N/A
2,AAP,39.75,2.445907,N/A
3,AAPL,272.15,4003.948444,N/A
4,ABBV,228.805,402.804647,N/A
...,...,...,...,...
500,YUM,154.9725,40.49844,N/A
501,ZBH,89.51,18.46259,N/A
502,ZBRA,246.75,13.32618,N/A
503,ZION,59.25,8.22803,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [48]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f''


######## Finnhub does not support batch API requests ########

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [19]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    portfolio_size = float(portfolio_size)
except ValueError:
    print("That's not a number! Please try again:")

In [82]:
position_size = portfolio_size/len(final_dataframe.index)

for i in range(0, len(final_dataframe.index)):
    if final_dataframe.loc[i, 'Stock Price'] == 'N/A':
        final_dataframe.loc[i, 'Number of Shares to Buy'] = 'N/A'
    else:
        final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/float(final_dataframe.loc[i, 'Stock Price']))

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,138.42,39.208695,14
1,AAL,15.795,10.733006,125
2,AAP,39.75,2.445907,49
3,AAPL,272.15,4003.948444,7
4,ABBV,228.805,402.804647,8
...,...,...,...,...
500,YUM,154.9725,40.49844,12
501,ZBH,89.51,18.46259,22
502,ZBRA,246.75,13.32618,8
503,ZION,59.25,8.22803,33


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [83]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

/var/folders/h9/w84pm_9x5qs91pvj4jj1p5fc0000gp/T/ipykernel_97929/817941762.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  final_dataframe.to_excel(writer, 'Recommended Trades', index = False)


### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [84]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [ ]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.close()

writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [85]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [86]:
writer.close()